In [78]:
#Remove NA
import pandas as pd
df=pd.read_csv("Real_Combine.csv")
df

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5,186.041667


In [79]:
df.fillna(df['PM 2.5'].mean(),inplace=True)

In [80]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    0
dtype: int64

In [81]:
!pip install keras-tuner

In [82]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [83]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1:]

In [84]:
X


,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2
...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5


In [85]:
y

,PM 2.5
0,219.720833
1,182.187500
2,154.037500
3,223.208333
4,200.645833
...,...
1088,288.416667
1089,256.833333
1090,169.000000
1091,186.041667


In [86]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [87]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='abc',
    project_name='Air Quality Index')

In [88]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [89]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [90]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 06s]
val_mean_absolute_error: 63.4583994547526

Best val_mean_absolute_error So Far: 47.8683713277181
Total elapsed time: 00h 00m 30s
INFO:tensorflow:Oracle triggered exit


In [91]:
tuner.results_summary()

Results summary
Results in abc\Air Quality Index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 2
units_0: 320
units_1: 288
learning_rate: 0.01
units_2: 96
units_3: 224
units_4: 288
units_5: 480
units_6: 320
units_7: 288
units_8: 256
units_9: 352
units_10: 192
units_11: 416
Score: 47.8683713277181
Trial summary
Hyperparameters:
num_layers: 10
units_0: 192
units_1: 352
learning_rate: 0.0001
units_2: 256
units_3: 320
units_4: 64
units_5: 352
units_6: 160
units_7: 128
units_8: 352
units_9: 416
units_10: 160
units_11: 160
units_12: 448
units_13: 384
units_14: 64
units_15: 64
units_16: 352
units_17: 480
units_18: 384
Score: 63.4583994547526
Trial summary
Hyperparameters:
num_layers: 14
units_0: 448
units_1: 352
learning_rate: 0.0001
units_2: 192
units_3: 96
units_4: 192
units_5: 288
units_6: 160
units_7: 64
units_8: 384
units_9: 320
units_10: 256
units_11: 416
units_12: 32
units_13: 32
Score: 63.490736643473305
Tr

In [92]:
tuner.get_best_models(num_models=2)